In [3]:
import xlwings

In [4]:
xlwings.sheets

Sheets([<Sheet [통합 문서1]Sheet1>])

In [5]:
xlwings.Range("A1")

<Range [통합 문서1]Sheet1!$A$1>

In [6]:
xlwings.Range("A1").expand()

<Range [통합 문서1]Sheet1!$A$1:$B$8>

In [10]:
total_rows = xlwings.Range("A1").expand().value
total_rows

[['message ', 'browser'],
 ['첫 로그', 'chrome'],
 ['두번째 로그', 'chrome'],
 ['세번째 로그', 'chrome'],
 ['로그 #4', 'chrome'],
 ['로그#5', 'chrome'],
 ['로그 #4', 'chrome'],
 ['로그 #4', 'chrome']]

In [11]:
column_names = total_rows[0]
column_names

['message ', 'browser']

In [14]:
for row in total_rows[1:]:
    row_dict = dict(zip(column_names, row))
    print(row_dict)

{'message ': '첫 로그', 'browser': 'chrome'}
{'message ': '두번째 로그', 'browser': 'chrome'}
{'message ': '세번째 로그', 'browser': 'chrome'}
{'message ': '로그 #4', 'browser': 'chrome'}
{'message ': '로그#5', 'browser': 'chrome'}
{'message ': '로그 #4', 'browser': 'chrome'}
{'message ': '로그 #4', 'browser': 'chrome'}


In [16]:
# list comprehension

rows_data=[
    dict(zip(column_names, row))
    for row in total_rows[1:]
]
rows_data

[{'message ': '첫 로그', 'browser': 'chrome'},
 {'message ': '두번째 로그', 'browser': 'chrome'},
 {'message ': '세번째 로그', 'browser': 'chrome'},
 {'message ': '로그 #4', 'browser': 'chrome'},
 {'message ': '로그#5', 'browser': 'chrome'},
 {'message ': '로그 #4', 'browser': 'chrome'},
 {'message ': '로그 #4', 'browser': 'chrome'}]

# 정리

In [18]:
def get_rows_data():
    total_rows = xlwings.Range("A1").expand().value
    column_names = total_rows[0]
    rows_data = [
        dict(zip(column_names, row))
        for row in total_rows[1:]
    ]
    return rows_data

get_rows_data()

[{'message ': '첫 로그', 'browser': 'chrome'},
 {'message ': '두번째 로그', 'browser': 'chrome'},
 {'message ': '세번째 로그', 'browser': 'chrome'},
 {'message ': '로그 #4', 'browser': 'chrome'},
 {'message ': '로그#5', 'browser': 'chrome'},
 {'message ': '로그 #4', 'browser': 'chrome'},
 {'message ': '로그 #4', 'browser': 'chrome'}]

# 데이터베이스 접속

In [23]:
DATABASE = {
    "host" : "127.0.0.1",
    "user" : "root",
    "password" : "1234",
    "db" : "myproj11_db",
    'charset' : "utf8",
}

In [28]:
import pymysql

connection = pymysql.connect(**DATABASE)
cursor = connection.cursor()
cursor.execute("SELECT * FROM log")
print(cursor.fetchall())
cursor.close()

()


# with 절

In [41]:
import pymysql
from contextlib import contextmanager

@contextmanager
def db_cursor():
    connection = pymysql.connect(**DATABASE)
    cursor = connection.cursor()
    try:
        yield cursor
    finally:
        cursor.close()  

In [42]:
with db_cursor() as cursor:
    cursor.execute("SELECT*FROM log")
    print(cursor.fetchall())

()


# Header

In [46]:
column_data = {'message ': '로그 #4', 'browser': 'chrome'}

with db_cursor()as cursor:
    for row in get_rows_data():
    sql = "INSERT INTO LOG(message, browser)VALUES(%(message)s, %(browser)s)"
    cursor.execute(sql, row_dict)

IndentationError: expected an indented block after 'for' statement on line 4 (Temp/ipykernel_7184/2748928157.py, line 5)

In [43]:
get_rows_data()

[{'message ': '첫 로그', 'browser': 'chrome'},
 {'message ': '두번째 로그', 'browser': 'chrome'},
 {'message ': '세번째 로그', 'browser': 'chrome'},
 {'message ': '로그 #4', 'browser': 'chrome'},
 {'message ': '로그#5', 'browser': 'chrome'},
 {'message ': '로그 #4', 'browser': 'chrome'},
 {'message ': '로그 #4', 'browser': 'chrome'}]

# SQL INJECTION공격의 예

In [69]:
with db_cursor() as cursor:
    log_id = '3 or 3'
    # sql = "SELECT * FROM LOG WHERE id = " + str(log_id)
    # sql = "SELECT * FROM LOG WHERE id = 3 or 3"
    sql = "SELECT * FROM LOG WHERE id = %s"
    cursor.execute(sql, [log_id])
#     while True:
#         row = cursor.fetchone()
#         print(row)
#         if row is None:
#             break
#         print(row)
    for row in cursor:
        print(row)

(3, '3로그', 'Chrome')


# 데이터 매핑의 예

In [74]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    rows_data = cursor.fetchall()
#     for row in cursor:
#         row[0] # id
#         row[1] # message
#         row[2] #chrome
#         print(row)
rows_data

((3, '3로그', 'Chrome'), (4, '로그#4', 'Chrome'), (5, '로그#5', 'Chrome'))

# dict 으로 변환

## namedtuple

In [92]:
from collections import namedtuple

Log = namedtuple('Log', 'id message browser')

In [93]:
(1,'message', 'chrome')

(1, 'message', 'chrome')

In [94]:
log = (1, 'message', 'chrome')
print(log[0])
print(log[1])
print(log[2])

1
message
chrome


In [95]:
print(log[0])
print(log[1])
print(log[2])

1
message
chrome


In [98]:
 log.id, log.message, log.browser

AttributeError: 'tuple' object has no attribute 'message'

In [81]:
rows_data

((3, '3로그', 'Chrome'), (4, '로그#4', 'Chrome'), (5, '로그#5', 'Chrome'))

In [90]:
with db_cursor() as cursor:
    cursor.execute("SELECT * FROM log")
    rows_data = [
        Log(*row)
        for now in cursor.fetchall()]

rows_data

[Log(id=5, message='로그#5', browser='Chrome'),
 Log(id=5, message='로그#5', browser='Chrome'),
 Log(id=5, message='로그#5', browser='Chrome')]